In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print(f'length: {len(text)}')

length: 1115394


In [6]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [11]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f'vocab size: {vocab_size}')


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [14]:
# create character to number encoder and decoder
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

def encode(s):
    return [stoi[c] for c in s]

def decode(indices):
    return ''.join(itos[i] for i in indices)

print(encode("hello there."))
print(decode(encode("hello there.")))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43, 8]
hello there.


In [19]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [20]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [21]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [22]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"input: {context}, target: {target}")

input: tensor([18]), target: 47
input: tensor([18, 47]), target: 56
input: tensor([18, 47, 56]), target: 57
input: tensor([18, 47, 56, 57]), target: 58
input: tensor([18, 47, 56, 57, 58]), target: 1
input: tensor([18, 47, 56, 57, 58,  1]), target: 15
input: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47
input: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58


In [24]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b][:t+1]
        target = yb[b][t]
        print(f"input: {context}, target: {target}")

inputs: 
torch.Size([4, 8])
tensor([[ 1, 58, 46, 39, 58,  1, 56, 43],
        [42,  6,  1, 50, 47, 49, 43,  1],
        [20, 17, 30, 25, 21, 27, 26, 17],
        [41, 39, 56, 43, 11,  0, 18, 53]])
targets:
torch.Size([4, 8])
tensor([[58, 46, 39, 58,  1, 56, 43, 41],
        [ 6,  1, 50, 47, 49, 43,  1, 53],
        [17, 30, 25, 21, 27, 26, 17, 10],
        [39, 56, 43, 11,  0, 18, 53, 56]])
input: tensor([1]), target: 58
input: tensor([ 1, 58]), target: 46
input: tensor([ 1, 58, 46]), target: 39
input: tensor([ 1, 58, 46, 39]), target: 58
input: tensor([ 1, 58, 46, 39, 58]), target: 1
input: tensor([ 1, 58, 46, 39, 58,  1]), target: 56
input: tensor([ 1, 58, 46, 39, 58,  1, 56]), target: 43
input: tensor([ 1, 58, 46, 39, 58,  1, 56, 43]), target: 41
input: tensor([42]), target: 6
input: tensor([42,  6]), target: 1
input: tensor([42,  6,  1]), target: 50
input: tensor([42,  6,  1, 50]), target: 47
input: tensor([42,  6,  1, 50, 47]), target: 49
input: tensor([42,  6,  1, 50, 47, 49]), t

In [26]:
print(xb)

tensor([[ 1, 58, 46, 39, 58,  1, 56, 43],
        [42,  6,  1, 50, 47, 49, 43,  1],
        [20, 17, 30, 25, 21, 27, 26, 17],
        [41, 39, 56, 43, 11,  0, 18, 53]])


In [42]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx
            
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)  
print(logits.shape)
print(loss)


print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(5.0851, grad_fn=<NllLossBackward0>)

J; vP'g.zcCxJbMHx!3aDo?!SyfgFx! HXWBnLw,3a&K$PWLSeDiRnh?QoMnhwUKOvkdUBidJngXWZ$GjvlcyuieAJ!dlCE3AJGc


In [46]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [62]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.5213851928710938


In [65]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


I at beredaven ireertiewnerinkin aleco Gon an: wicand, ff fe

S:
Purcheacadeepative?
my hame w.
Topu
